In [1]:
import sqlite3
import sqlite_vec
import os

In [41]:
# Add url_picture_dewey column to users table
try:
    # Check if column already exists
    cursor_check = db.execute("PRAGMA table_info(users)")
    columns = [column[1] for column in cursor_check.fetchall()]
    
    if "url_picture_dewey" not in columns:
        db.execute("ALTER TABLE users ADD COLUMN url_picture_dewey TEXT")
        db.commit()
        print("Successfully added 'url_picture_dewey' column to 'users' table")
    else:
        print("Column 'url_picture_dewey' already exists in 'users' table")
except Exception as e:
    print(f"Error adding column: {e}")
    db.rollback()

Successfully added 'url_picture_dewey' column to 'users' table


In [ ]:
# Create program_user table for many-to-many relationship
try:
    # Check if table already exists
    cursor_check = db.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='program_user'")
    table_exists = cursor_check.fetchone()
    
    if not table_exists:
        db.execute("""
        CREATE TABLE program_user (
            program_id INTEGER,
            user_id INTEGER,
            PRIMARY KEY (program_id, user_id),
            FOREIGN KEY (program_id) REFERENCES programs(id),
            FOREIGN KEY (user_id) REFERENCES users(id)
        )
        """)
        db.commit()
        print("Successfully created 'program_user' table")
    else:
        print("Table 'program_user' already exists")
except Exception as e:
    print(f"Error creating table: {e}")
    db.rollback()

Successfully created 'program_user' table


In [112]:
# Create programs table
try:
    # Check if table already exists
    cursor_check = db.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='programs'")
    table_exists = cursor_check.fetchone()
    
    if not table_exists:
        db.execute("""
        CREATE TABLE programs (
            id INTEGER PRIMARY KEY,
            name TEXT,
            url TEXT
        )
        """)
        db.commit()
        print("Successfully created 'programs' table")
    else:
        print("Table 'programs' already exists")
except Exception as e:
    print(f"Error creating table: {e}")
    db.rollback()


Successfully created 'programs' table


In [117]:
cursor_check = db.execute("SELECT name FROM sqlite_master")
table_exists = cursor_check.fetchall()
for table in table_exists:
    print(table[0])

users
publications
sqlite_sequence
publication_user_mapping
sqlite_autoindex_publication_user_mapping_1
publications_vec_bge_m3_info
sqlite_autoindex_publications_vec_bge_m3_info_1
publications_vec_bge_m3_chunks
publications_vec_bge_m3_rowids
publications_vec_bge_m3_vector_chunks00
sqlite_autoindex_publications_vec_bge_m3_vector_chunks00_1
publications_vec_all_MiniLM_L6_v2_info
sqlite_autoindex_publications_vec_all_MiniLM_L6_v2_info_1
publications_vec_all_MiniLM_L6_v2_chunks
publications_vec_all_MiniLM_L6_v2_rowids
publications_vec_all_MiniLM_L6_v2_vector_chunks00
sqlite_autoindex_publications_vec_all_MiniLM_L6_v2_vector_chunks00_1
publications_vec_indobert_info
sqlite_autoindex_publications_vec_indobert_info_1
publications_vec_indobert_chunks
publications_vec_indobert_rowids
publications_vec_indobert_vector_chunks00
sqlite_autoindex_publications_vec_indobert_vector_chunks00_1
publications_vec_bge_m3
publications_vec_all_MiniLM_L6_v2
publications_vec_indobert
program_user
sqlite_autoin

In [2]:
# Connect to the SQLite database
try:
    db = sqlite3.connect('../../data/cleaned_with_bge_m3.db')
    print("Connected to SQLite database successfully")
    
    # Check table info
    cursor_check = db.execute("PRAGMA table_info(users)")
    print(cursor_check.fetchall())
except Exception as e:
    print(f"Error connecting to database: {e}")

Connected to SQLite database successfully
[(0, 'id', 'INTEGER', 0, None, 1), (1, 'scholar_id', 'TEXT', 0, None, 0), (2, 'name', 'TEXT', 0, None, 0), (3, 'original_names', 'TEXT', 0, None, 0), (4, 'interests', 'TEXT', 0, None, 0), (5, 'url_picture', 'TEXT', 0, None, 0), (6, 'url_picture_dewey', 'TEXT', 0, None, 0)]


In [91]:
import mysql.connector

database = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)
cursor = database.cursor()
cursor.execute("USE nlp_thesis_similarity")


In [6]:
# Query to get all programs from the programs table
cursor.execute("SELECT * FROM programs")

# Fetch all results
programs = cursor.fetchall()

# Print all programs
print("All programs in the programs table:")
for program in programs:
    print(program)

All programs in the programs table:
(1, 'Civil Engineering', 'https://civil.petra.ac.id/')
(2, 'Architecture', 'https://architecture.petra.ac.id/')
(3, 'Architecture of Sustainable Housing and Real Estate', 'https://shre.petra.ac.id/')
(4, 'Master’s Program in Civil Engineering', 'https://mcivil.petra.ac.id/')
(5, 'Master’s Program in Architecture', 'https://mars.petra.ac.id/')
(6, 'Doctoral Program in Civil Engineering', 'https://doctoral-civil.petra.ac.id/')
(7, 'Electrical Engineering', 'https://electrical.petra.ac.id/')
(8, 'Internet of Things', 'https://iot.petra.ac.id/')
(9, 'Sustainable Mechanical Engineering and Design', 'https://smed.petra.ac.id/')
(10, 'Automotive', 'https://automotive.petra.ac.id/')
(11, 'Industrial Engineering', 'https://industrial.petra.ac.id/')
(12, 'Global Logistics and Supply Chain', 'https://glsc.petra.ac.id/')
(13, 'International Business Engineering', 'https://ibe.petra.ac.id/')
(14, 'Informatics', 'https://informatics.petra.ac.id/')
(15, 'Business I

In [97]:
import re

def get_dosen(name):
    # Clean up the name string to improve matching
    name = name.strip()
    # Check if the name contains a comma (indicating possible title or additional info)
    if "," in name:
        name = name.split(",")[0].strip()
    # Remove any titles that might affect matching
    titles = ["Dr.", "Prof.", "Ir.", "S.T.", "M.T.", "M.Eng.", "Ph.D.", "M.Sc.", "M.Eng.Sc."]
    for title in titles:
        name = name.replace(title, "").strip()
    # Remove multiple spaces
    name = re.sub(' +', ' ', name)
    cursor_sqlite = db.execute(
        "SELECT * FROM users WHERE name LIKE ?",
        (f"%{name}%",),
    )
    result = cursor_sqlite.fetchall()
    if result:
        return result
    else:
        return None
    
def update_dosen_photo(dosen_id, photo_path):
    db.execute(
        "UPDATE users SET url_picture_dewey = ? WHERE id = ?",
        (photo_path, dosen_id)
    )
    db.commit()
    print(f"Updated photo for Dosen ID {dosen_id} with path {photo_path}")
    
def insert_program_user(program_id, user_id):
    try:
        db.execute(
            "INSERT INTO program_user (program_id, user_id) VALUES (?, ?)",
            (program_id, user_id)
        )
        db.commit()
        print(f"Inserted program_user with program_id {program_id} and user_id {user_id}")
    except sqlite3.IntegrityError as e:
        print(f"Integrity error: {e}")
    except Exception as e:
        print(f"Error inserting into program_user: {e}")

In [98]:
import requests
from bs4 import BeautifulSoup
import time
from requests.exceptions import Timeout, ConnectionError, RequestException

# Configure retry parameters
MAX_RETRIES = 3
BASE_TIMEOUT = 15  # seconds
RETRY_DELAY = 2  # seconds

# i = 0
for program_id, program_name, program_url in programs:
    # i += 1
    
    if not program_url:  # Skip if URL is None or empty
        print(f"Skipping {program_name} - No URL available")
        continue
    
    program_url = f"{program_url}/#abt"
    print(f"Scraping {program_name} from {program_url}")
    
    # Implement retry logic
    retry_count = 0
    success = False
    
    while retry_count < MAX_RETRIES and not success:
        try:
            # Increase timeout with each retry
            current_timeout = BASE_TIMEOUT * (retry_count + 1)
            print(f"Attempt {retry_count + 1} with timeout {current_timeout}s")
            
            response = requests.get(program_url, timeout=current_timeout)
            success = True  # If we get here, the request succeeded
            
            if response.status_code == 200:
                # Store the HTML content
                html_content = response.text
                # Create BeautifulSoup object for parsing
                soup = BeautifulSoup(html_content, 'html.parser')
                text_content = soup.get_text(separator=' ', strip=True)
                
                # Try to find author wrapper div if it exists
                author_div = soup.find_all('div', class_='wrapperauthor')
                print(f"Found {len(author_div)} author divs")
                for div in author_div:
                    # Try to find the author name in h5 element
                    author_name = div.find('h5')
                    if author_name:
                        author_text = author_name.get_text(strip=True)
                        dosen_data = get_dosen(author_text)
                        # print(div.parent)
                        img = (div.parent).find('img')
                        img_src = img.get('data-src') if img else None
                        # print(img_src)
                        
                        if img_src is None:
                            img_src = img.get('src') if img else None
                        
                        if dosen_data is not None and len(dosen_data) == 1:
                            # print(dosen_data[0][-1])
                            # print(dosen_data[0][0])
                            insert_program_user(program_id, dosen_data[0][0])
                            update_dosen_photo(dosen_data[0][0], img_src)
                            print(f"Updated photo for {author_text}")
                    else:
                        print("No h5 element found in author div")
            else:
                print(f"Error: Received status code {response.status_code} for {program_url}")
                
        except Timeout as e:
            retry_count += 1
            if retry_count < MAX_RETRIES:
                print(f"Timeout error: {e}. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print(f"Failed after {MAX_RETRIES} attempts: {e}")
                
        except ConnectionError as e:
            retry_count += 1
            if retry_count < MAX_RETRIES:
                print(f"Connection error: {e}. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print(f"Failed after {MAX_RETRIES} attempts: {e}")
                
        except RequestException as e:
            print(f"Request error: {e}")
            retry_count = MAX_RETRIES  # Don't retry for other request exceptions
            
        except Exception as e:
            print(f"Unexpected error processing {program_name}: {e}")
            break
    
    # Add a short delay between programs to avoid overwhelming the server
    time.sleep(1)
    
    # if i == 1: break

Scraping Civil Engineering from https://civil.petra.ac.id//#abt
Attempt 1 with timeout 15s
Found 24 author divs
Inserted program_user with program_id 1 and user_id 1509
Updated photo for Dosen ID 1509 with path http://civil.petra.ac.id/wp-content/uploads/2023/09/cropped-wftjong-min-scaled-1.jpg
Updated photo for Wong Foek Tjong, S.T., M.T., Ph.D.
Inserted program_user with program_id 1 and user_id 1499
Updated photo for Dosen ID 1499 with path http://civil.petra.ac.id/wp-content/uploads/2023/09/cropped-willy.husada-removebg-preview.jpg
Updated photo for Willy Husada, S.T., M.T., M.Sc.
Inserted program_user with program_id 1 and user_id 383
Updated photo for Dosen ID 383 with path http://civil.petra.ac.id/wp-content/uploads/2023/09/cropped-djwantoro-min-scaled-1.jpg
Updated photo for Prof. Dr. Ir. Djwantoro Hardjito, M.Eng.
Inserted program_user with program_id 1 and user_id 1085
Updated photo for Dosen ID 1085 with path http://civil.petra.ac.id/wp-content/uploads/2023/09/cropped-paul-m

In [9]:
tables = db.execute("""
    SELECT * FROM users WHERE name LIKE '%Gregorius Satia Budhi%'
""").fetchall()

print(tables)

[(535, 'a6-Tvz8AAAAJ', 'Gregorius Satia Budhi', None, 'Artificial Intelligence;Machine Learning;Data and Text Mining;Sentiment Analysis', 'https://scholar.google.com/citations?view_op=medium_photo&user=a6-Tvz8AAAAJ', 'http://dsa.petra.ac.id/wp-content/uploads/2023/08/cropped-Dr.-Gregorius-Satiabudhi-S.T.-M-scaled-1.jpg')]


In [6]:
tables = db.execute("""
    SELECT users.*, programs.id as program_id, programs.name as program_name, users.url_picture_dewey, users.url_picture
    FROM users 
    JOIN program_user ON users.id = program_user.user_id 
    JOIN programs ON program_user.program_id = programs.id
    WHERE users.name LIKE '%njo%'
""").fetchall()
print(len(tables))
# print(tables)

for row in tables:
    print(row) 

6
(716, None, 'Iwan Njoto Sandjaja', None, None, None, 'https://dsa.petra.ac.id/wp-content/uploads/2023/10/Iwan-Njoto.webp', 14, 'Informatics', 'https://dsa.petra.ac.id/wp-content/uploads/2023/10/Iwan-Njoto.webp', None)
(716, None, 'Iwan Njoto Sandjaja', None, None, None, 'https://dsa.petra.ac.id/wp-content/uploads/2023/10/Iwan-Njoto.webp', 15, 'Business Information System', 'https://dsa.petra.ac.id/wp-content/uploads/2023/10/Iwan-Njoto.webp', None)
(716, None, 'Iwan Njoto Sandjaja', None, None, None, 'https://dsa.petra.ac.id/wp-content/uploads/2023/10/Iwan-Njoto.webp', 16, 'Data Science and Analytics', 'https://dsa.petra.ac.id/wp-content/uploads/2023/10/Iwan-Njoto.webp', None)
(1034, 'Kf2J1igAAAAJ', 'Njo Anastasia', None, 'Property/ Business Valuation;Corporate/ Personal Finance;Financial Behaviour;Real Estate Finance', 'https://scholar.google.com/citations?view_op=medium_photo&user=Kf2J1igAAAAJ', 'https://dm.petra.ac.id/wp-content/uploads/2025/04/Untitled-design.jpg', 22, 'Finance an

In [119]:
# Clear any unread results to avoid "Unread result found" error
while cursor.nextset():
    pass

# Query to get all programs from the programs table
cursor.execute("SELECT * FROM programs")

# Fetch all results
programs = cursor.fetchall()

# Insert programs from MySQL to SQLite if they don't exist
for program in programs:
    try:
        # Check if program already exists in SQLite
        existing_program = db.execute(
            "SELECT * FROM programs WHERE id = ?",
            (program[0],)
        ).fetchone()
        
        if not existing_program:
            # Insert program into SQLite database
            db.execute(
                "INSERT INTO programs (id, name, url) VALUES (?, ?, ?)",
                (program[0], program[1], program[2])
            )
            print(f"Inserted program: {program[0]} - {program[1]}")
        else:
            print(f"Program already exists: {program[0]} - {program[1]}")
            
        # Print all program details
        print(f"Program ID: {program[0]}, Name: {program[1]}, URL: {program[2]}")
    except Exception as e:
        print(f"Error processing program {program}: {e}")
        
# Commit changes to SQLite database
db.commit()



Inserted program: 1 - Civil Engineering
Program ID: 1, Name: Civil Engineering, URL: https://civil.petra.ac.id/
Inserted program: 2 - Architecture
Program ID: 2, Name: Architecture, URL: https://architecture.petra.ac.id/
Inserted program: 3 - Architecture of Sustainable Housing and Real Estate
Program ID: 3, Name: Architecture of Sustainable Housing and Real Estate, URL: https://shre.petra.ac.id/
Inserted program: 4 - Master’s Program in Civil Engineering
Program ID: 4, Name: Master’s Program in Civil Engineering, URL: https://mcivil.petra.ac.id/
Inserted program: 5 - Master’s Program in Architecture
Program ID: 5, Name: Master’s Program in Architecture, URL: https://mars.petra.ac.id/
Inserted program: 6 - Doctoral Program in Civil Engineering
Program ID: 6, Name: Doctoral Program in Civil Engineering, URL: https://doctoral-civil.petra.ac.id/
Inserted program: 7 - Electrical Engineering
Program ID: 7, Name: Electrical Engineering, URL: https://electrical.petra.ac.id/
Inserted program: 

In [ ]:
print(db.execute("SELECT * FROM users WHERE name LIKE '%Liliana%'").fetchall())



[(854, None, 'Liliana', None, None, None, None), (855, '0piCBRQAAAAJ', 'Liliana .', None, 'Teknologi Game;Computer Graphics;Computer Vision', 'https://scholar.google.com/citations?view_op=medium_photo&user=0piCBRQAAAAJ', None), (856, None, 'Liliana Prajoga', None, None, None, None)]
